In [42]:
import librosa, numpy as np
from model.mobilenet_mfcc import mobilenet_v2_mfcc
import torch, time, os
from sklearn.metrics import f1_score

# wav 파일을 MFCC로 변환하는 함수
def wav_to_mfcc(file_path, n_mfcc=40, max_len=100):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    
    # MFCC의 길이를 max_len으로 맞추기
    if mfcc.shape[1] < max_len:
        mfcc = np.pad(mfcc, ((0, 0), (0, max_len - mfcc.shape[1])), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

def predict_single_file(file_path, model, class_names, device='cuda'):
    """단일 WAV 파일에 대한 예측을 수행합니다."""
    # 오디오 파일 로드 및 MFCC 변환
    mfcc = wav_to_mfcc(file_path)
    
    # MFCC를 모델 입력 형식으로 변환
    mfcc = np.expand_dims(mfcc, axis=(0, 1))  # (1, 1, n_mfcc, max_len) 형태로 변환
    mfcc_tensor = torch.FloatTensor(mfcc).to(device)
    
    # 예측 수행
    model.eval()
    with torch.no_grad():
        outputs = model(mfcc_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
    
    # 결과 반환
    predicted_label = class_names[predicted_class]
    confidence = probabilities[0][predicted_class].item()
    
    return predicted_label, confidence

In [43]:
binary_class = ['home_noise', 'not_noise']
class_names = ['가구끄는소리',
 '강아지짓는소리',
 '고양이우는소리',
 '골프퍼팅(골굴리는소리)',
 '드럼세탁기소리',
 '런닝머신에서뛰는소리',
 '망치질소리',
 '문여닫는소리',
 '바이올린연주소리',
 '샤워할때물소리',
 '식기세척기소리',
 '아이들발걸음소리',
 '어른발걸음소리',
 '진공청소기소리',
 '통돌이세탁기소리',
 '피아노연주소리',
 '화장실물내리는소리']

# 모델 로딩
model = mobilenet_v2_mfcc(num_classes=len(class_names))
model.load_state_dict(torch.load('model/mobilenet_mfcc_model2424.pth'))
# GPU 사용 가능 시 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [44]:
## 1. 노이즈 유무 예측
# stime = time.time()
count = 0
correct_count = 0
y_true = []
y_pred = []
target_dir = './test_data/noise_or_not'
for directory in os.listdir(target_dir):
    for file in os.listdir(os.path.join(target_dir, directory)):
        if file.endswith('wav'):
            wavpath = os.path.join(target_dir, directory, file)
            count += 1
            _, confidence = predict_single_file(wavpath, model, class_names, device)
            predicted_label = 'home_noise' if confidence > 0.6 else 'not_noise'
            # print(time.time()-stime)
            y_true.append(binary_class.index(directory))
            y_pred.append(binary_class.index(predicted_label))
            if predicted_label == directory:
                correct_count += 1
                res = 'OK'
            else:
                res = 'FAIL'
            print(f"파일명: {file}, 실제: {directory}, 예측: {predicted_label}, 결과: {res}")
print("------------------------------")
print(f"총 파일 수: {count}개, 올바른 예측: {correct_count}개")

# F1 스코어 계산
f1 = f1_score(y_true, y_pred)
print(f'F1 Score: {f1:.4f}')

파일명: normal3159.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal1069.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal2478.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal2947.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal495.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal1646.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal3287.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal3422.wav, 실제: not_noise, 예측: home_noise, 결과: FAIL
파일명: normal1224.wav, 실제: not_noise, 예측: home_noise, 결과: FAIL
파일명: normal1382.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal1993.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal1586.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal1385.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal608.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal661.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal2088.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal414.wav, 실제: not_noise, 예측: not_noise, 결과: OK
파일명: normal1

In [45]:
## 2. 노이즈 종류 예측
# stime = time.time()
stime = time.time()
count = 0
correct_count = 0
y_true = []
y_pred = []
target_dir = './test_data/classification'
for directory in os.listdir(target_dir):
    for file in os.listdir(os.path.join(target_dir, directory)):
        if file.endswith('wav'):
            wavpath = os.path.join(target_dir, directory, file)
            count += 1
            predicted_label, confidence = predict_single_file(wavpath, model, class_names, device)
            # print(time.time()-stime)
            y_true.append(class_names.index(directory))
            y_pred.append(class_names.index(predicted_label))
            if predicted_label == directory:
                correct_count += 1
                res = 'OK'
            else:
                res = 'FAIL'
            print(f"파일명: {file}, 실제: {directory}, 예측: {predicted_label}, 결과: {res}")
                
print("------------------------------")
print(f"총 파일 수: {count}개, 올바른 예측: {correct_count}개")

# F1 스코어 계산
f1 = f1_score(y_true, y_pred, average='macro')
print(f'F1 Score: {f1:.4f}')

파일명: N-10_221001_A_4_a_14123_1.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_220915_A_4_a_15025_0.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_221001_A_4_a_13908_0.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_220918_A_4_a_14235_1.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_220920_A_4_a_14266_2.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_221011_A_4_a_14785_0.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_221008_A_4_a_14593_1.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_221007_A_4_a_14704_0.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_221011_A_4_a_14773_7.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_221010_A_4_a_14759_1.wav, 실제: 바이올린연주소리, 예측: 바이올린연주소리, 결과: OK
파일명: N-10_220923_A_3_a_07755_0.wav, 실제: 화장실물내리는소리, 예측: 화장실물내리는소리, 결과: OK
파일명: N-10_220901_A_3_a_08367_0.wav, 실제: 화장실물내리는소리, 예측: 화장실물내리는소리, 결과: OK
파일명: N-10_220925_A_3_a_08131_0.wav, 실제: 화장실물내리는소리, 예측: 화장실물내리는소리, 결과: OK
파일명: N-10_220928_A_3_a_08335_0.wav, 실제: 화장실물내리는소리, 예측: 화장실물내리는소리, 결과: O